In [181]:
import json
import numpy as np
import cvxpy as cp

In [197]:
with open("C:/Users/morea/OneDrive/Documents/2A/REOP/KIRO/small_1.json", 'r') as file:
    data = json.load(file)

vehicles = {}
for i in range(len(data["vehicles"])):
    vehicles[i+1] = data["vehicles"][i]["type"]

constraints = []
for i in range(len(data["constraints"])):
    constraints.append(data["constraints"][i])

lag = {}
for i in range(len(data["shops"])):
    lag[data["shops"][i]["name"]] = data["shops"][i]["resequencing_lag"]


parameters = data["parameters"]

In [198]:
lag

{'body': 3, 'paint': 3, 'assembly': 0}

In [199]:
constraints

[{'max_vehicles': 2,
  'cost': 5000,
  'id': 1,
  'window_size': 3,
  'vehicles': [3,
   6,
   7,
   17,
   18,
   19,
   21,
   23,
   24,
   26,
   27,
   28,
   29,
   30,
   31,
   33,
   35,
   36,
   37,
   39,
   40,
   43,
   44,
   45,
   49,
   50,
   51,
   54],
  'type': 'rolling_window',
  'shop': 'assembly'},
 {'max_vehicles': 5,
  'cost': 2000,
  'id': 2,
  'window_size': 9,
  'vehicles': [3,
   6,
   17,
   18,
   23,
   26,
   27,
   28,
   29,
   30,
   31,
   33,
   35,
   37,
   40,
   43,
   45,
   51,
   54],
  'type': 'rolling_window',
  'shop': 'assembly'},
 {'max_vehicles': 5,
  'cost': 700,
  'min_vehicles': 1,
  'id': 3,
  'vehicles': [1,
   2,
   3,
   4,
   5,
   7,
   8,
   9,
   10,
   12,
   14,
   15,
   17,
   18,
   19,
   21,
   22,
   24,
   25,
   26,
   28,
   29,
   30,
   31,
   32,
   34,
   35,
   37,
   38,
   40,
   41,
   42,
   43,
   47,
   48,
   49,
   51,
   54],
  'type': 'batch_size',
  'shop': 'body'},
 {'max_vehicles': 3,
  'cost':

In [200]:
rolling_window_constraints=[]
lot_change_constraints=[]
batch_size_constraints=[]

for cont in constraints:
    if cont['type']=='rolling_window':
        rolling_window_constraints.append(cont)
    elif cont['type']=='batch_size':
        batch_size_constraints.append(cont)
    elif cont['type']=='lot_change':
        lot_change_constraints.append(cont)


In [201]:
R=len(rolling_window_constraints)
S=3
n= len(vehicles.keys()) #nb vehicule
shops={'body':0,'paint':1,'assembly':2}

In [202]:
lin_constraints = []

objective_func=0

# resequencing constraints !!!

# Create two scalar optimization variables.
x = cp.Variable((S,n,n),integer=True)
y = cp.Variable((S,n),integer=True)
Z = cp.Variable((S,R,n),integer=True)



Contraintes 

In [203]:
lin_constraints.append(x>=0)
lin_constraints.append(x<=1)
lin_constraints.append(Z>=0)
lin_constraints.append(y>=0)

# permutation
for s in range(S):
    for t in range(n):
        sum=0
        for v in range(n):
            sum+=x[s][v][t]
        lin_constraints.append(sum==1)
    for v in range(n):
        sum=0
        for t in range(n):
            sum+=x[s][v][t]
        lin_constraints.append(sum==1)


In [204]:
# resequencing
for s in range(S-1):
    for v in range(n):
        sum=0
        for t in range(n):
            sum+=t*x[s][v][t]
            sum-=t*x[s+1][v][t]
        sum-=lag[list(shops.keys())[s]]
        lin_constraints.append(y[s][v]>=sum)


In [205]:
SUM = np.zeros((S,R,n))


In [206]:
C_r = 0
for r in range(R):
    cont=rolling_window_constraints[r]
    w_r = cont["window_size"]
    c_r = cont["cost"]
    Mr = cont["max_vehicles"]
    Vr= cont["vehicles"]
    s=shops[cont['shop']]
    sum=0
    for t in range(n-w_r+1):
        for t_ in range(t,t+w_r):
            for v in Vr:
                sum+=x[s][v-1][t_]
                
        sum-=Mr
        C_r+=c_r*Z[s][r][t]
        lin_constraints.append(Z[s][r][t]>=sum)

objective_func+=C_r

In [207]:
lin_constraints.append(Z>=SUM)


In [208]:
      
            

    elif cont['type']=='batch_size':


    elif cont['type']=='lot_change':

    zs0 = cp.Variable((0,len(Rs[0]),n))
    zs1 = cp.Variable((1,len(Rs[1]),n))
    zs2 = cp.Variable((2,len(Rs[2]),n))
    g = cp.Variable((S,n,n))

# Create two constraints.




SyntaxError: invalid syntax (1273209441.py, line 1)

In [209]:

# Form objective.
obj = cp.Minimize(objective_func)

# Form and solve problem.
prob = cp.Problem(obj, lin_constraints)
prob.solve()

# The optimal dual variable (Lagrange multiplier) for
# a constraint is stored in constraint.dual_value.


C:\Users\morea\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\cvxpy\problems\problem.py:164: UserWarning: Constraint #465 contains too many subexpressions. Consider vectorizing your CVXPY code to speed up compilation.
  warnings.warn(f"Constraint #{i} contains too many subexpressions. "
C:\Users\morea\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\cvxpy\problems\problem.py:164: UserWarning: Constraint #466 contains too many subexpressions. Consider vectorizing your CVXPY code to speed up compilation.
  warnings.warn(f"Constraint #{i} contains too many subexpressions. "
C:\Users\morea\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\cvxpy\problems\problem.py:164: UserWarning: Constraint #467 contains too many subexpressions. Consider vectorizing your CVXPY code 

KeyboardInterrupt: 

In [195]:
prob.value

-0.0

In [196]:
x.value

array([[[-0., -0., -0., -0.,  1.],
        [-0.,  1., -0., -0., -0.],
        [-0., -0.,  1., -0., -0.],
        [-0., -0., -0.,  1., -0.],
        [ 1., -0., -0., -0., -0.]],

       [[-0., -0., -0.,  1., -0.],
        [-0., -0.,  1., -0., -0.],
        [ 1., -0., -0., -0., -0.],
        [-0.,  1., -0., -0., -0.],
        [-0., -0., -0., -0.,  1.]],

       [[-0., -0., -0., -0.,  1.],
        [-0., -0.,  1., -0., -0.],
        [-0.,  1., -0., -0., -0.],
        [ 1., -0., -0., -0., -0.],
        [-0., -0., -0.,  1., -0.]]])